In [3]:
from ultralytics import YOLO

model = YOLO('/Users/filippovarini/Desktop/train/weights/best.pt')

In [4]:
# Train the model
results = model.val(data='/Volumes/TOSHIBA EXT/data/yolov8/data_config.yaml', split='test')

Ultralytics YOLOv8.0.237 🚀 Python-3.10.13 torch-2.1.2 CPU (Intel Core(TM) i7-9750H 2.60GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients


val: Scanning /Volumes/TOSHIBA EXT/data/yolov8/test/labels... 170 images, 41 backgrounds, 0 corrupt: 100%|██████████| 211/211 [00:01<00:00, 193.10it/s]

val: New cache created: /Volumes/TOSHIBA EXT/data/yolov8/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:25<00:00,  1.80s/it]


                   all        211        210      0.941       0.91      0.947      0.635
Speed: 3.4ms preprocess, 102.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val


In [9]:
import os
all_images = os.listdir('/Volumes/TOSHIBA EXT/data/yolov8/test/images')
all_images = [f'/Volumes/TOSHIBA EXT/data/yolov8/test/images/{f}' for f in all_images if f.endswith('.jpg')]

inference_results = model(all_images[:2])


0: 640x640 (no detections), 1: 640x640 1 shark, 225.9ms
Speed: 4.9ms preprocess, 112.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)


In [13]:
# Process results list
for result in inference_results:
    boxes = result.boxes  # Boxes object for bbox outputs
    boxes = boxes.xyxy  # Boxes object for bbox outputs (normalized)
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    print(boxes, masks, keypoints, probs)



tensor([], size=(0, 4)) None None None
tensor([[442.1426, 461.3682, 502.3386, 493.5381]]) None None None


# Tracker

In [9]:
from collections import defaultdict

import cv2
import numpy as np

from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('/vol/biomedic3/bglocker/ugproj2324/fv220/dev/old/shark_locator_tests/runs/detect/yolov8m_mvd2/best.pt')
video_folder = '/vol/biomedic3/bglocker/ugproj2324/fv220/datasets/validation/val1/videos/'
output_folder = '/vol/biomedic3/bglocker/ugproj2324/fv220/datasets/annotations/test/'
CONF_TRESHOLD = 0.2

videos = ['easy1']

for video in videos:
    video_path = video_folder + video + '.mp4'
    cap = cv2.VideoCapture(video_path)

    # Store the track history
    track_history = defaultdict(lambda: [])

    filename = video_path.split('/')[-1]
    FILE_OUTPUT = f"{output_folder}{filename.split('.')[0]}_tracked.avi"
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    out = cv2.VideoWriter(FILE_OUTPUT, cv2.VideoWriter_fourcc('M','J','P','G'),
                            30, (frame_width, frame_height))

    
    i = 0
    # Loop through the video frames
    while cap.isOpened():
        i += 1
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, persist=True, conf=CONF_TRESHOLD)

            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            tracks = results[0].boxes.id
            track_ids = tracks.int().cpu().tolist() if tracks is not None else []

            if len(boxes) != len(track_ids):
                print(f'frame: {i}, results: {zip(boxes, track_ids)}')

            # Visualize the results on the frame
            annotated_frame = results[0].plot()

            # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 90 tracks for 90 frames
                    track.pop(0)

                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

            # # Display the annotated frame
            # cv2.imshow('YOLOv8 Tracking', annotated_frame)

            # # Break the loop if 'q' is pressed
            # if cv2.waitKey(1) & 0xFF == ord('q'):
            #     break
                
            out.write(annotated_frame)
        else:
            # Break the loop if the end of the video is reached
            break

    # Release the video capture object and close the display window
    cap.release()
    out.release()


0: 384x640 (no detections), 15.7ms
Speed: 4.5ms preprocess, 15.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.3ms
Speed: 1.5ms preprocess, 7.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 shark, 6.8ms
Speed: 1.7ms preprocess, 6.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
frame: 3, results: <zip object at 0x7f87530c7680>

0: 384x640 1 shark, 6.3ms
Speed: 1.3ms preprocess, 6.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
frame: 4, results: <zip object at 0x7f86c17bcc80>

0: 384x640 1 shark, 7.0ms
Speed: 3.3ms preprocess, 7.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
frame: 5, results: <zip object at 0x7f8752f990c0>

0: 384x640 1 shark, 6.0ms
Speed: 1.2ms preprocess, 6.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
frame: 6, results: <zip object at 0x7f8752f68900>

0: 384x640 1 shark, 6.0ms
Speed: 1.2ms prep

In [3]:
cap.release()
cv2.destroyAllWindows()

: 

'/vol/biomedic3/bglocker/ugproj2324/fv220/dev/shark_locator_tests/runs/detect/train3/easy1.avi'

In [3]:
import cv2

output_folder = '/vol/biomedic3/bglocker/ugproj2324/fv220/dev/shark_locator_tests/runs/detect/train3/'
avi_file = output_folder + 'easy1_tracked.avi'

#  turn avi file into mp4 with cv2
cap = cv2.VideoCapture(avi_file)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
FILE_OUTPUT = avi_file.split('.')[0] + '.mp4'
out = cv2.VideoWriter(FILE_OUTPUT, cv2.VideoWriter_fourcc('M','J','P','G'),
                            30, (frame_width, frame_height))
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()
    if success:
        out.write(frame)
    else:
        break
cap.release()
out.release()

OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [8]:
avi_file

'/vol/biomedic3/bglocker/ugproj2324/fv220/dev/shark_locator_tests/runs/detect/train3/easy1.avi'